In [1]:
import pandas as pd
import ast
from tqdm import tqdm
tqdm.pandas()

In [2]:
# Load Data
df_yake_claims = pd.read_json('/mnt/hdd01/PATSTAT Working Directory/PATSTAT/cleantech_epo_text_data_pivot_cleaned_yake.json')

In [ ]:
df_yake_claims.head()

In [3]:
# Iterate over rows in dataframe
for index, row in tqdm(df_yake_claims.iterrows()):
    # Cast row 'keywords_yake_claims' column to string
    row['keywords_yake_claims'] = str(row['keywords_yake_claims']).lower()
    # Check if keywords_yake_claims column starts with "[[[" and ends with "]]]""
    if row['keywords_yake_claims'].startswith('[[[') and row['keywords_yake_claims'].endswith(']]]'):
        # Remove first "[" and last "]" from keywords_yake_claims column
        row['keywords_yake_claims'] = row['keywords_yake_claims'][1:-1]
    # Cast row 'keywords_yake_claims' column to list
    row['keywords_yake_claims'] = ast.literal_eval(row['keywords_yake_claims'])
    # Assign modified 'keywords_yake_claims' list to temporary variable
    keywords_yake_temp = row['keywords_yake_claims']
    # Assign temporary variable to 'keywords_yake_claims' column
    df_yake_claims.at[index, 'keywords_yake_claims'] = keywords_yake_temp

182369it [00:25, 7142.91it/s]


In [4]:
df_yake_claims['keywords_yake_claims_exploded'] = ''
# Iterate over rows in dataframe
for index,row in tqdm(df_yake_claims.iterrows()):
    # Check if 'keywords_yake_claims' column is not a list
    if not isinstance(row['keywords_yake_claims'], list):
        continue
    # Check if 'keywords_yake_claims' column is an empty list or contains only empty lists
    if not any(keyword for keyword in row['keywords_yake_claims']):
        continue
    # Iterate over keywords in 'keywords_yake_claims' column and append to 'keywords_yake_claims_exploded' column
    else:
        for keyword in row['keywords_yake_claims']:
            df_yake_claims.at[index, 'keywords_yake_claims_exploded'] += keyword[0] + ', '

182369it [00:44, 4095.22it/s]


In [5]:
# Cast 'keywords_yake_claims_exploded' column to list
df_yake_claims['keywords_yake_claims_exploded'] = df_yake_claims['keywords_yake_claims_exploded'].str.split(', ')

In [6]:
# Delete last item in 'keywords_yake_claims_exploded' cells if empty string
df_yake_claims['keywords_yake_claims_exploded'] = df_yake_claims['keywords_yake_claims_exploded'].progress_apply(lambda x: x[:-1] if x and x[-1] == '' else x)

100%|██████████| 182369/182369 [00:00<00:00, 1229970.10it/s]


In [7]:
keywords_list = []
min_yake_conf = 0.05

# Iterate over rows in dataframe
for index,row in tqdm(df_yake_claims.iterrows()):
    # Check if 'keywords_yake_claims' column is not a list
    if not isinstance(row['keywords_yake_claims'], list):
        continue
    # Check if 'keywords_yake_claims' column is an empty list or contains only empty lists
    if not any(keyword for keyword in row['keywords_yake_claims']):
        continue
    # Iterate over keywords in 'keywords_yake_claims' column and append to 'keywords_yake_claims_exploded' column
    else:
        for keyword in row['keywords_yake_claims']:
            if keyword[1] <= min_yake_conf:
                keywords_list.append(keyword[0])

182369it [00:06, 29717.08it/s]


In [9]:
# Only consider alphanumeric characters in keywords
keywords_list = [
    keyword for keyword in keywords_list 
    if all(word.isalnum() for word in keyword.split())
]

# Postprocessing

In [10]:
# Generate unique list of keywords
keywords_list_unique = list(set(keywords_list))
df_keywords_list_unique = pd.DataFrame(keywords_list_unique, columns=['keyword_yake'])

In [12]:
# Count absolute frequency of each keyword
from collections import Counter

# Step 1: Flatten the list of keywords into a single list
flattened_keywords = [keyword for keyword_list in tqdm(df_yake_claims['keywords_yake_claims_exploded']) for keyword in keyword_list]

# Step 2: Use collections.Counter to count the occurrences of each keyword
keyword_counts = Counter(flattened_keywords)

# Step 3: Convert the keyword counts to a DataFrame
count_df = pd.DataFrame(keyword_counts.items(), columns=['keyword_yake', 'abs_frequency'])

# Step 4: Merge the count_df with df_keywords_list_unique on the 'keyword_yake' column
df_keywords_list_unique = df_keywords_list_unique.merge(count_df, on='keyword_yake', how='left')

100%|██████████| 182369/182369 [00:00<00:00, 1561469.92it/s]


In [ ]:
### MAYBE I NEED TO REDO THIS WITH FULL TEXT CLAIMS INSTEAD OF JUST THE YAKE KEYWORDS

# Calculate document frequency by dividing absolute frequency by len of df_yake_claims
# df_keywords_list_unique['doc_frequency'] = df_keywords_list_unique['abs_frequency'] / len(df_yake_claims)

In [15]:
# Prune dataframe by document frequency and absolute frequency
min_abs_frequency = 5
max_abs_frequency = 1000
# max_doc_frequency = 0.3

df_keywords_list_unique_pruned = df_keywords_list_unique[(df_keywords_list_unique['abs_frequency'] >= min_abs_frequency) & (df_keywords_list_unique['abs_frequency'] <= max_abs_frequency)]

# Reset index
df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

In [ ]:
# Only consider n-grams with n >= 2 - NOT CONSIDERED FOR NOW
# min_ngram_length = 2

# # Generate copy of df_keywords_list_unique_pruned
# df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# def filter_keywords(keyword_string):
#     # Split the keyword string into individual keywords
#     keywords = keyword_string.split(', ')
#     # Filter out keywords with fewer than 2 words
#     filtered_keywords = [keyword for keyword in keywords if len(keyword.split()) >= min_ngram_length]
#     # Join the filtered keywords back into a string
#     return ', '.join(filtered_keywords)

# # Create a copy of df_keywords_list_unique_pruned to avoid SettingWithCopyWarning
# df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# # Now use .loc to modify the 'keyword_yake' column
# df_keywords_list_unique_pruned.loc[:, 'keyword_yake'] = df_keywords_list_unique_pruned['keyword_yake'].progress_apply(filter_keywords)

# # Delete rows with empty 'keyword_yake' column
# df_keywords_list_unique_pruned = df_keywords_list_unique_pruned[df_keywords_list_unique_pruned['keyword_yake'] != '']

# # Reset index
# df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

In [16]:
import nltk
from nltk.corpus import stopwords

# Download stopwords
# nltk.download('stopwords')

# Generate copy of df_keywords_list_unique_pruned
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# Delete all stopwords from 'keyword_yake' column
df_keywords_list_unique_pruned.loc[:, 'keyword_yake'] = df_keywords_list_unique_pruned['keyword_yake'].progress_apply(
    lambda x: '' if x in stopwords.words('english') else x
)

# Remove empty rows
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned[df_keywords_list_unique_pruned['keyword_yake'] != '']

# Reset index
df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

100%|██████████| 26871/26871 [00:01<00:00, 19766.22it/s]


In [17]:
### MAYBE I NEED TO REDO THIS PART WITH THE CLAIM TEXTS INSTEAD OF THE KEYWORDS - NOT CONSIDERED FOR NOW

import spacy

# Download spacy model
nlp = spacy.load('en_core_web_sm')

# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned_pos = df_keywords_list_unique_pruned.copy()

# Perform part-of-speech tagging on the 'keyword_yake' column 
# and save the POS tags in a new column 'keyword_yake_pos'
df_keywords_list_unique_pruned_pos['keyword_yake_pos'] = df_keywords_list_unique_pruned_pos['keyword_yake'].progress_apply(
    lambda x: [token.pos_ for token in nlp(x)]
)

/home/thiesen/Documents/Cleantech_Concepts/venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 26871/26871 [01:02<00:00, 427.77it/s]


In [20]:
# Filter out all keywords that do not contain NOUN, PRON or PROPN in their POS tags
df_keywords_list_unique_pruned_pos = df_keywords_list_unique_pruned_pos[df_keywords_list_unique_pruned_pos['keyword_yake_pos'].progress_apply(
    lambda x: any(pos in ['NOUN', 'PRON', 'PROPN'] for pos in x)
)]

# Reset index
df_keywords_list_unique_pruned_pos.reset_index(drop=True, inplace=True)

100%|██████████| 26871/26871 [00:00<00:00, 486685.39it/s]


In [ ]:
# NOT CONSIDERED FOR NOW
# import re

# # Function to check if a word contains at least three consecutive capital letters (filter out all abbreviations)
# def has_three_consecutive_caps(word):
#     return re.search(r'[A-Z]{3,}', word) is not None

# # Function to check if any word in an n-gram contains at least three consecutive capital letters
# def check_ngram(ngram):
#     return any(has_three_consecutive_caps(word) for word in ngram.split())

# # Use the DataFrame `apply` method to filter rows
# df_keywords_list_unique_pruned_pos_filtered = df_keywords_list_unique_pruned_pos[~df_keywords_list_unique_pruned_pos['keyword_yake'].progress_apply(check_ngram)]

In [ ]:
# NOT CONSIDERED FOR NOW
# # Test to filter out brown corpus words
# from nltk.corpus import brown
# # nltk.download('brown')

# # Step 1: Pre-calculate the set of Brown corpus words
# brown_words_set = set(brown.words())

# # Function to check if any word in a keyword phrase exists in the Brown corpus
# def check_brown_words(phrase):
#     return any(word in brown_words_set for word in phrase.split())

# # Step 2: Use progress_apply with the optimized function
# df_keywords_list_unique_pruned_pos_filtered_brown = df_keywords_list_unique_pruned_pos_filtered[
#     df_keywords_list_unique_pruned_pos_filtered['keyword_yake'].progress_apply(check_brown_words)
# ]

# Aggregate Keywords on Patent Id and CPC Classification

In [21]:
rows_list = []

# Iterate over rows in dataframe
# for index, row in tqdm(df_yake_claims_cpc.iterrows()):
for index,row in tqdm(df_yake_claims.iterrows()):
    if row['keywords_yake_claims_exploded'] == []:
        continue
    if not any(keyword for keyword in row['keywords_yake_claims_exploded']):
        continue
    # Iterate over keywords in 'keywords_yake' column and append patent_id and cpc information
    else:
        for keyword in row['keywords_yake_claims_exploded']:
            # EPO
            rows_list.append({'publn_nr': row['publn_nr'],
                              'cpc_class_symbol': row['cpc_class_symbol'],
                              'keyword_yake_claims': keyword})

df_claims_keywords = pd.DataFrame(rows_list)

182369it [00:11, 16527.90it/s]


In [22]:
# Aggregate df_claims_keywords on 'keyword_yake_claims' column
df_claims_keywords_agg = df_claims_keywords.groupby('keyword_yake_claims').agg(
    publn_nr_list = pd.NamedAgg(column='publn_nr', aggfunc=list),
    cpc_class_symbol_list = pd.NamedAgg(column='cpc_class_symbol', aggfunc=list)
)

In [23]:
# Remove duplicates from 'cpc_class_symbol_list'
df_claims_keywords_agg['cpc_class_symbol_list'] = df_claims_keywords_agg['cpc_class_symbol_list'].progress_apply(lambda x: list(set(x)))

100%|██████████| 602746/602746 [00:02<00:00, 287177.01it/s] 


In [26]:
# Merge df_claims_keywords_agg with df_keywords_list_unique_embeddings on 'keyword_yake_claims' column and generate new dataframe
df_claims_keywords_list = pd.merge(df_keywords_list_unique_pruned_pos, df_claims_keywords_agg, left_on='keyword_yake', right_on='keyword_yake_claims', how='left')

# Keyword Embedding

In [31]:
from sentence_transformers import SentenceTransformer
import torch

In [32]:
model_climatebert = SentenceTransformer('climatebert/distilroberta-base-climate-f')
model_bertforpatents = SentenceTransformer('anferico/bert-for-patents')
model_patentsberta = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')

No sentence-transformers model found with name /home/thiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f. Creating a new one with MEAN pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at /home/thiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
No sentence-transformers model found with name /home/thiesen/.cache/torch/sentence_transformers/anferico_bert-for-patents. Creating a new one with MEAN pooling.


In [33]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available: {}".format(torch.cuda.get_device_name(0)))
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU available: NVIDIA RTX A4500


In [36]:
# Generate copy of df_claims_keywords_list
df_keywords_list_unique_embeddings = df_claims_keywords_list.copy()

# Perform sentence embedding on the 'keyword_yake' (PatentsView) or 'keywords_yake_claims' (EPO) column
df_keywords_list_unique_embeddings['keyword_yake_patentsberta_embedding'] = df_keywords_list_unique_embeddings['keyword_yake'].progress_apply(
    lambda x: model_patentsberta.encode(x)
)

df_keywords_list_unique_embeddings['keyword_yake_climatebert_embedding'] = df_keywords_list_unique_embeddings['keyword_yake'].progress_apply(
    lambda x: model_climatebert.encode(x)
)

df_keywords_list_unique_embeddings['keyword_yake_bertforpatents_embedding'] = df_keywords_list_unique_embeddings['keyword_yake'].progress_apply(
    lambda x: model_bertforpatents.encode(x)
)

100%|██████████| 23105/23105 [04:55<00:00, 78.29it/s]


In [37]:
df_keywords_list_unique_embeddings.head()

,keyword_yake,abs_frequency,keyword_yake_pos,publn_nr_list,cpc_class_symbol_list,keyword_yake_patentsberta_embedding,keyword_yake_climatebert_embedding,keyword_yake_bertforpatents_embedding
0,acetic anhydride,9,"[ADJ, NOUN]","[380911, 381988, 810028, 1132384, 1220829, 137...","[['Y02P 20/54', 'Y02P 20/584'], ['Y02P 20/5...","[-0.061444506, -1.0947561, -0.2616166, -0.1942...","[0.0051785745, 0.21960728, -0.05591166, -0.108...","[0.4696103, 0.05110017, -0.06434366, 0.2766463..."
1,detectors,7,[NOUN],"[64008, 280925, 1683698, 1842205, 2294406, 262...","[['Y02D 30/70'], ['Y02T 10/62', 'Y02T 10/70...","[-0.12924775, -0.4697809, -0.36494324, 0.32934...","[0.015357403, 0.17548731, 0.03343723, 0.090828...","[-0.40965247, 0.7753821, -1.0722224, 0.0085351..."
2,digital subscriber line,6,"[PROPN, NOUN, NOUN]","[1189422, 1843610, 2832086, 2907270, 2907282, ...","[['Y02D 30/50'], ['Y02D 30/50', 'Y02D 30/70...","[-0.051423278, -0.27481222, -0.1730148, -0.450...","[0.014533296, -0.046291605, -0.06059263, -0.12...","[0.0066918074, 0.60324126, -0.11662749, -1.302..."
3,grain,42,[NOUN],"[62173, 94114, 267633, 462232, 514118, 586415,...","[['Y02W 30/91'], ['Y02E 30/30'], ['Y02P 10/...","[-0.20005703, -0.2817135, -0.24730219, 0.08159...","[-0.03290707, 0.0750718, 0.060500715, 0.080330...","[-0.38387752, -0.28565928, 0.11016483, -0.9580..."
4,processors,27,[NOUN],"[517558, 1387273, 1653332, 1916802, 2005659, 2...","[['Y02T 50/40'], ['Y02A 90/10'], ['Y02D 10/...","[-0.027116872, -0.34918648, -0.37371352, 0.364...","[-0.014848103, 0.07489975, -0.006813975, -0.02...","[-0.19758111, -0.15376149, -0.85155207, -0.059..."


In [38]:
# Save dataframe to json - Make sure to use correct directory (PatentsView vs. EPO)
df_keywords_list_unique_embeddings.to_json('/mnt/hdd01/PATSTAT Working Directory/PATSTAT/cleantech_epo_text_data_pivot_cleaned_yake_embeddings_processed.json', orient='records')

In [39]:
df_keywords_list_unique_embeddings.to_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/df_keywords_y02_ep_embeddings_processed.json', orient='records')

# Dimensionality Reduction

In [ ]:
# Load data if necessary
df_keywords_list_unique_embeddings = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings.json')

## UMAP

In [ ]:
import umap
import numpy as np
umap_dim = 2
reducer = umap.UMAP(random_state=42, n_components=umap_dim)

In [ ]:
# Stack all the embeddings into a 2D array
embeddings_patentsberta = np.vstack(df_keywords_list_unique_embeddings['keyword_yake_patentsberta_embedding'].values)

# Perform UMAP dimensionality reduction
umap_embeddings_patentsberta = reducer.fit_transform(embeddings_patentsberta)

# Assign the reduced dimension embeddings back to new DataFrame columns
df_keywords_list_unique_embeddings_umap['keyword_yake_patentsberta_embedding_umap'] = list(umap_embeddings_patentsberta)

## t-SNE

In [ ]:
import numpy as np
from openTSNE import TSNE
from sklearn.model_selection import train_test_split

In [ ]:
# Generate train/test split
patentsberta_x_train, patentsberta_x_test = train_test_split(df_keywords_list_unique_embeddings['keyword_yake_patentsberta_embedding'].tolist(), test_size=0.2, random_state=42)
patentsberta_x_train_np = np.array(patentsberta_x_train)
patentsberta_x_test_np = np.array(patentsberta_x_test)

In [ ]:
tsne = TSNE(
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    n_iter=1000,
    verbose=True,
)

In [ ]:
# Perform t-SNE on training data
patentsberta_embedding_train = tsne.fit(patentsberta_x_train_np)

In [ ]:
patentsberta_embedding_test = patentsberta_embedding_train.transform(patentsberta_x_test_np)

In [ ]:
df_keywords_list_unique_embeddings_tsne = df_keywords_list_unique_embeddings.copy()

patentsberta_embedding = np.concatenate((patentsberta_embedding_train, patentsberta_embedding_test), axis=0)
df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne'] = patentsberta_embedding.tolist()
df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne_x'] = df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne'].apply(lambda x: x[0])
df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne_y'] = df_keywords_list_unique_embeddings_tsne['keyword_yake_patentsberta_embedding_tsne'].apply(lambda x: x[1])

# Clustering

In [ ]:
# Import HDBSCAN
import hdbscan

In [ ]:
# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_embeddings_hdbscan = df_keywords_list_unique_embeddings_tsne.copy()

# Perform HDBSCAN clustering on the UMAP coordinates
clusterer_patentsberta = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=1).fit(df_keywords_list_unique_embeddings_hdbscan[['keyword_yake_patentsberta_embedding_tsne_x', 'keyword_yake_patentsberta_embedding_tsne_y']])

# Assign the cluster labels back to the DataFrame
df_keywords_list_unique_embeddings_hdbscan['patentsberta_cluster'] = clusterer_patentsberta.labels_

# Erase all rows with cluster -1
df_keywords_list_unique_embeddings_hdbscan = df_keywords_list_unique_embeddings_hdbscan[df_keywords_list_unique_embeddings_hdbscan['patentsberta_cluster'] != -1]

In [ ]:
df_keywords_list_unique_embeddings_hdbscan.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings_tsne_hdbscan.json', orient='records')

# Plotting

In [ ]:
import plotly.express as px

In [ ]:
# Plot the UMAP embeddings in three figures, color coded by cluster
fig_patentsberta = px.scatter(
    df_keywords_list_unique_embeddings_hdbscan, 
    x='keyword_yake_patentsberta_embedding_tsne_x', 
    y='keyword_yake_patentsberta_embedding_tsne_y', 
    color='patentsberta_cluster', 
    hover_data=['patentsberta_cluster', 'keyword_yake'], 
    title='Cleantech Yake Keywords - HDBSCAN, TSNE, PatentSBERTa',
    height=800,  # Adjust as needed
    width=800    # Adjust as needed
)
# Display the figures
fig_patentsberta.show()

# fig_patentsberta.write_html('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings_tsne.html')

# Name Cluster

In [ ]:
from transformers import pipeline, set_seed

In [ ]:
# Build dataframe of keywords and their corresponding clusters, where each row is a cluster with list of keywords
df_keywords_clusters_patentsberta = df_keywords_list_unique_embeddings_hdbscan.groupby('patentsberta_cluster')['keyword_yake'].apply(list).reset_index(name='keywords')

In [ ]:
# Initialize a text-generation pipeline with Flan-T5-large
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = pipeline('text2text-generation', model='google/flan-t5-large', device=device)
set_seed(42)

In [ ]:
def generate_cluster_name(keywords):
    # Ensure the keywords are in a list format
    keywords = keywords.split(', ') if isinstance(keywords, str) else keywords
    # Select only the first 1500 keywords from the list
    selected_keywords = keywords[:2000]
    # Join the selected keywords into a string format
    keywords_str = ', '.join(selected_keywords)
    # Create a prompt from the selected keywords
    # prompt = f"Based on the following keywords, come up with a specific, precise and short topic name: {keywords_str}"
    prompt = f"Generate a concise and descriptive common theme or category for a cluster containing the following keywords: {keywords_str}." # The name should be in title case and should not exceed three words."
    # Doesn't work at all -> only focuses on electric vehicle innovation# prompt = f"Given the keywords: {keywords_str}, provide a succinct cluster name similar to how 'Electric Vehicle Innovation' represents keywords like 'battery technology, electric motor, charging infrastructure'."
    # prompt = f"Identify a common theme or category for the following keywords: {keywords_str}. Provide a concise, descriptive name for this theme or category."
    # prompt = f"The keywords {keywords_str} all belong to the category: _____"
    # Generate a response using the GPT-3 model
    response = generator(prompt, max_length=10, do_sample=True, temperature=0.8)[0]['generated_text']
    # Extract the cluster name from the response
    cluster_name = response
    return cluster_name

# Apply the function to the 'keywords' column to generate cluster names
df_keywords_clusters_patentsberta['cluster_name'] = df_keywords_clusters_patentsberta['keywords'].progress_apply(generate_cluster_name)

In [ ]:
# Merge df_keywords_clusters_patentsberta with df_keywords_list_unique_pruned_embeddings_hdbscan on 'patentsberta_cluster'
df_keywords_list_unique_embeddings_cluster = pd.merge(df_keywords_list_unique_embeddings_hdbscan, df_keywords_clusters_patentsberta[['patentsberta_cluster', 'cluster_name']], on='patentsberta_cluster', how='left')

In [ ]:
# Plotting
fig_patentsberta_cluster = px.scatter(
    df_keywords_list_unique_embeddings_cluster, 
    x='keyword_yake_patentsberta_embedding_tsne_x', 
    y='keyword_yake_patentsberta_embedding_tsne_y', 
    color='cluster_name', 
    hover_data=['patentsberta_cluster', 'keyword_yake'], 
    title='Cleantech Yake Keywords - HDBSCAN, TSNE, PatentSBERTa',
    height=800,  # Adjust as needed
    width=1200    # Adjust as needed
)

# Display the figures
fig_patentsberta_cluster.show()

In [ ]:
# Arrange column order
df_keywords_clusters_patentsberta = df_keywords_clusters_patentsberta[['patentsberta_cluster', 'cluster_name', 'keywords']]

In [ ]:
df_keywords_clusters_patentsberta.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_clusters_patentsberta.json', orient='records')

In [ ]:
df_keywords_list_unique_embeddings_cluster.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_embeddings_tsne_hdbscan_cluster.json', orient='records')